In [24]:
import pandas as pd
import pickle
import numpy as np
from tqdm import tqdm
import string
import zipfile

#### Скачивание данных по ссылке из Яндекс-Диска

In [25]:
!curl -L $(yadisk-direct https://disk.yandex.ru/d/3klCc5D4_3Y-IA) -o data/test.csv.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  289M  100  289M    0     0  6550k      0  0:00:45  0:00:45 --:--:-- 3938k0:00:11  0:00:38 4589k


In [26]:
zip_file_path = './data/test.csv.zip'
extract_to_directory = './data/'


In [27]:
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to_directory)

In [28]:
data = pd.read_csv('./data/test.csv')

In [29]:
data.shape

(432158, 3)

In [6]:
data.topic.unique()

array(['Мир', 'Экономика', 'Интернет и СМИ', 'Спорт', 'Культура',
       'Наука и техника'], dtype=object)

In [7]:
data.topic.nunique()

6

In [8]:
data.topic.value_counts()

Мир                136621
Экономика           79528
Спорт               64413
Культура            53797
Наука и техника     53136
Интернет и СМИ      44663
Name: topic, dtype: int64

#### Задача multiclass classification - одному объекту соотвествует только одно из 6-ти возможных значений таргета.

In [9]:
data.head()

,title,text,topic
0,Британцы отмечают двухлетие смерти Дианы,"<?xml version=""1.0"" encoding=""utf-8""?>\n<conve...",Мир
1,Еще одно землетрясение в Турции: один человек ...,"<?xml version=""1.0"" encoding=""utf-8""?>\n<conve...",Мир
2,Российские национал-большевики убирают террито...,"<?xml version=""1.0"" encoding=""utf-8""?>\n<conve...",Мир
3,Киргизия ведет бои на границах с Таджикистаном...,"<?xml version=""1.0"" encoding=""utf-8""?>\n<conve...",Мир
4,Литва засудила участников переворота 91 года,"<?xml version=""1.0"" encoding=""utf-8""?>\n<conve...",Мир


#### Закодируем значения таргетов

In [10]:
def convert_target(df: pd.DataFrame) -> None:
    target_value = {}
    for i, item in enumerate(df.topic.unique()):
        target_value[item] = i
    
    df['target'] = df.topic.apply(lambda x: target_value[x])
    
    

In [11]:
convert_target(data)
    

#### Из-за вычислительной сложности работать буду только с заголовками

In [12]:
data = data.drop(['text'], axis=1)

### Токенизация и лемматизация

In [13]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/alexg/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [14]:
from nltk.corpus import stopwords
import pymorphy2

In [15]:
morph = pymorphy2.MorphAnalyzer()

In [16]:
stopw = stopwords.words('russian')  # стоп-слова

In [17]:
punct = list(string.punctuation) + ['«', '»'] # знаки пунктуации

In [18]:
from razdel import tokenize  # токенизатор

In [19]:
def tokenize_text(item: str) -> str:
    toks = [word.text for word in tokenize(item)]
    # не включаем токены - знаки пунктуации и стоп слова
    toks = [token.lower() for token in toks if (token not in punct) and (token not in stopw)]
    toks = ' '.join(toks)
    return toks
    

In [20]:
data['title'] = data.title.apply(lambda x: tokenize_text(x))

In [22]:
data.to_pickle('./data/proc_data.pickle')

In [23]:
data.head()

,title,topic,target
0,британцы отмечают двухлетие смерти дианы,Мир,0
1,еще одно землетрясение турции человек погиб ок...,Мир,0
2,российские национал-большевики убирают террито...,Мир,0
3,киргизия ведет бои границах таджикистаном узбе...,Мир,0
4,литва засудила участников переворота 91 года,Мир,0


In [43]:
def convert_(chunk: pd.Series) -> list[tuple[int, str]:
    chunk = chunk.to_list()
    # (индекс, текст) в списке
    out = [(i, chunk[i]) for i in range(len(chunk))]
    return out

In [48]:
def lemmotize(item: tuple[int, str]) -> tuple[int, str]:
    idx = item[0]
    # лемметазация текста
    text = item[1].split(' ')
    pymorphy_results = list(map(lambda x: morph.parse(x), text))
    lemm_text = [res[0].normal_form for res in pymorphy_results]
    
    return (idx, lemm_text)
    

In [49]:
import multiprocessing

In [50]:
titles = convert_(data.title)

#### Распараллеливание вычислений, задавая 10 процессов

In [52]:
with multiprocessing.Pool(processes=10) as pool:
    lemm_titles = pool.map(lemmotize, titles)

In [56]:
def convert_todict(lemm_titles: list[tuple[int, str]] -> dict[int, str]:
    lemm_dict_titles = {}
    
    for item in lemm_titles:
        lemm_dict_titles[item[0]] = ' '.join(item[1])
        
    return lemm_dict_titles
        

In [57]:
lemm_dict_titles = convert_todict(lemm_titles)

In [60]:
def save_obj(obj: object, name: str) -> None:
    with open(name, 'wb') as f:
        pickle.dump(obj, f)

In [62]:
save_obj(lemm_dict_titles, './data/lemm_titles.pkl')